In [2]:
"""
TODO:
* features to add: PPG, allowed PPG, steals, turnovers, rebounds, assists, FG%, home/away/neutral, conference (1-whatever), title in last 4 yrs?
* refactor getStats to optomize for speed
+ https://adeshpande3.github.io/Applying-Machine-Learning-to-March-Madness
"""

# import modules
import pandas as pd
import numpy as np

## Preparations: Load & Manipulate Data

In [15]:
# load raw datasets
dfTeamNames = pd.read_csv("../data/raw/WTeams.csv")
dfSeasonInfo = pd.read_csv("../data/raw/WSeasons.csv")
dfSeasonResultsDet = pd.read_csv("../data/raw/WRegularSeasonDetailedResults.csv")

dfSeasonResultsDet.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Pred
46337,2018,132,3195,68,3239,58,H,0,24,58,...,9,13,14,18,8,19,5,5,19,1.0
46338,2018,132,3251,60,3421,42,H,0,21,56,...,12,15,8,22,4,10,3,1,14,1.0
46339,2018,132,3311,69,3372,65,N,0,24,47,...,12,14,16,20,15,16,9,2,17,1.0
46340,2018,132,3343,63,3335,34,N,0,21,54,...,6,9,7,24,7,9,4,4,19,1.0
46341,2018,132,3384,66,3352,56,H,0,19,46,...,5,10,13,25,11,18,9,0,24,1.0


## Preparations: Make Features & Target

In [10]:
# allSeasons = list(dfSeasonInfo["Season"])
selectSeasons = list(range(2014,2019))

In [30]:
# prepare dfStats aka features
def getStats(df, seasons):
    
    
    def winRatio(df):
        
        dfWinRatio = dfTeamNames[["TeamID"]].copy()
        dfWinRatio = dfWinRatio.reindex(columns = ["TeamID","Wins","Losses","WinRatio"], fill_value = 0) # create columns and init w/ 0s
        # find indx of winTeam & loseTeam and add one to their column counts
        # confirmed that each team plays ~30 games total in a season (https://en.wikipedia.org/wiki/2018–19_NCAA_Division_I_women%27s_basketball_season)
        for index, row in df.iterrows():
            dfWinRatio.loc[dfWinRatio["TeamID"] == row["WTeamID"], "Wins"] += 1
            dfWinRatio.loc[dfWinRatio["TeamID"] == row["LTeamID"], "Losses"] += 1
        
        dfWinRatio["WinRatio"] = round(dfWinRatio["Wins"] / (dfWinRatio["Wins"] + dfWinRatio["Losses"]), 3)
        
        return dfWinRatio
    
    
    def ppg(df):
        dfPpg = dfTeamNames[["TeamID"]].copy()
        dfPpg = dfPpg.reindex(columns = ["TeamID", "TotalPoints", "TotalGames", "PPG"], fill_value = 0) # create columns and init w/ 0s

        for index, row in df.iterrows():
            dfPpg.loc[dfPpg["TeamID"] == row["WTeamID"], "TotalPoints"] += row["WScore"]
            dfPpg.loc[dfPpg["TeamID"] == row["WTeamID"], "TotalGames"] += 1
            dfPpg.loc[dfPpg["TeamID"] == row["LTeamID"], "TotalPoints"] += row["LScore"]
            dfPpg.loc[dfPpg["TeamID"] == row["LTeamID"], "TotalGames"] += 1

        dfPpg["PPG"] = round(dfPpg["TotalPoints"] / dfPpg["TotalGames"], 3)

        return dfPpg
    
    
    def allowPpg(df):
        dfAllowPpg = dfTeamNames[["TeamID"]].copy()
        dfAllowPpg = dfAllowPpg.reindex(columns = ["TeamID", "TotalPoints", "TotalGames", "AllowedPPG"], fill_value = 0) # create columns and init w/ 0s

        for index, row in df.iterrows():
            dfAllowPpg.loc[dfAllowPpg["TeamID"] == row["WTeamID"], "TotalPoints"] += row["LScore"]
            dfAllowPpg.loc[dfAllowPpg["TeamID"] == row["WTeamID"], "TotalGames"] += 1
            dfAllowPpg.loc[dfAllowPpg["TeamID"] == row["LTeamID"], "TotalPoints"] += row["WScore"]
            dfAllowPpg.loc[dfAllowPpg["TeamID"] == row["LTeamID"], "TotalGames"] += 1

        dfAllowPpg["AllowedPPG"] = round(dfAllowPpg["TotalPoints"] / dfAllowPpg["TotalGames"], 3)

        return dfAllowPpg
    
    
    def steals(df):
        return
    
    def turnovers(df):
        return
    
    def rebounds(df):
        return
    
    def assists(df):
        return
    
    def fgPer(df):
        return
    
    def homeAwayNeutral(df):
        return
    
    def conf(df):
        return
    
    dfStats = pd.DataFrame([])
    for season in seasons:
        dfStatsOneSeason = dfTeamNames[["TeamID"]].copy()
        dfStatsOneSeason["Season"] = [season] * len(dfStatsOneSeason.iloc[:, 0])
        
        dfWinRatio = winRatio(df.loc[df["Season"] == season])
        dfWinRatio.drop(columns = ["TeamID", "Wins", "Losses"], axis = 1, inplace = True)
        
        dfPpg = ppg(df.loc[df["Season"] == season])
        dfPpg.drop(columns = ["TeamID", "TotalPoints", "TotalGames"], axis = 1, inplace = True)
        
        dfAllowPpg = allowPpg(df.loc[df["Season"] == season])
        dfAllowPpg.drop(columns = ["TeamID", "TotalPoints", "TotalGames"], axis = 1, inplace = True)
        
#         dfStatsOneSeason = pd.merge(dfStatsOneSeason, dfWinRatio, on = ["TeamID"])
        dfStatsOneSeason = pd.concat([dfStatsOneSeason, dfWinRatio, dfPpg, dfAllowPpg], axis = 1)
        dfStats = pd.concat([dfStats, dfStatsOneSeason], axis = 0)
        
    return dfStats
    
dfStats = getStats(dfSeasonResultsDet, selectSeasons)
dfStats.to_csv("../data/interim/TeamStats.csv", index = None, header = True)
dfStats.head()

,TeamID,Season,WinRatio,PPG,AllowedPPG
0,3101,2014,0.478,70.000,69.913
1,3102,2014,0.034,53.621,76.724
2,3103,2014,0.710,82.742,73.677
3,3104,2014,0.467,64.800,68.867
4,3105,2014,0.200,65.833,76.900
